In [ ]:
import pandas as pd 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/twitter_csv')

In [ ]:
df

In [ ]:
import re

In [ ]:
df['Content']

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF" 
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
df['Content'] = df['Content'].apply(remove_emoji)

In [ ]:
def remove_urls (tweet):
    tweet = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags=re.MULTILINE)
    return(tweet)

In [ ]:
df['Content'] = df['Content'].apply(remove_urls)

In [ ]:
df['Content']

In [ ]:
def get_tweet(tweets):
    tweet = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", tweets)
    return tweet

In [ ]:
df['Content'] = df['Content'].apply(get_tweet)

In [ ]:
df['Content']

In [ ]:
df['Content'][10000]

In [ ]:
def remove_num(tweets):
  tweet=re.sub("(\s\d+)","",tweets)
  return tweet

In [ ]:
df['Content'] = df['Content'].apply(remove_num)

In [ ]:
lines = []
for i in df['Content']:
    lines.append(i)
print(lines[-1])

In [ ]:
data = ""

for i in lines:
    data = ' '. join(lines)

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
    

In [ ]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)
data[:500]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
import numpy as np

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

In [ ]:
print(tokenizer.index_word)

In [ ]:
sequence = []
for i in range(3,len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequence.append(words)
print(len(sequence))
sequence = np.array(sequence)
sequence[:10]

In [ ]:
X = []
y = []

for i in sequence:
  X.append(i[:3])
  y.append(i[3])

X = np.array(X)
y = np.array(y)

In [ ]:
print(X[:10])
print(y[:10])

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y = to_categorical(y,num_classes=10000)

In [ ]:
y[:5]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(10000,64,input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(10000, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy')

In [ ]:
model.fit(X,y,epochs=100,batch_size=128)

In [ ]:
model.save('next_word.h5')